In [11]:
import importlib
import importlib
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [12]:

def dynamic_import(module_name, class_name):
    # Dynamically import the module
    module = importlib.import_module(module_name)
    # Get the class or object from the module
    return getattr(module, class_name)

# Dynamically import Sequential from tensorflow.keras.models
Sequential = dynamic_import('tensorflow.keras.models', 'Sequential')

# Dynamically import Dense from tensorflow.keras.layers
Dense = dynamic_import('tensorflow.keras.layers', 'Dense')

# Example usage of Sequential and Dense
model = Sequential()
model.add(Dense(64, input_dim=32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

c:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,177 (8.50 KB)

 Trainable params: 2,177 (8.50 KB)

 Non-trainable params: 0 (0.00 B)

None


In [13]:
import numpy as np
def load_data():
    X = np.load("static/uploads/data/X.npy")
    y = np.load("static/uploads/data/y.npy")
    X = X[0:1000]
    y = y[0:1000]
    
    return X, y
def load_weights():
    W1,b1 = np.load('weights/w1.npy'), np.load('weights/b1.npy')
    W2,b2 = np.load('weights/w2.npy'), np.load('weights/b2.npy')
    W3,b3 = np.load('weights/w3.npy'), np.load('weights/b3.npy')
    return W1,b1,W2,b2,W3,b3


def plot_data(X,y):
    m, n = X.shape
    fig, axes = plt.subplots(4,4, figsize=(4,4))
    fig.tight_layout(pad=0.1)

    for i, ax in enumerate(axes.flat):
        # Sélectionner des indices aléatoires
        random_index =  np.random.randint(0,m)
        
        # Sélectionner les lignes correspondant aux indices aléatoires et
        # redimensionner l'image
        X_random_reshaped =  X[random_index].reshape(20,20)
        
        # Afficher l'image
        ax.imshow(X_random_reshaped)
        
        # Afficher l'étiquette au-dessus de l'image
        ax.set_title(y[random_index, 0])
        ax.set_axis_off()


In [14]:
def train_model():
    # df=pd.read_csv('static/uploads/champs.csv')
    # plt.plot(df['name'])
    X,y=load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #plotthe data before trainng
    plot_data(X_train,y_train)
    
    return X_train,y_train,X_test,y_test

In [15]:
#implimentation using tensorflow
def train_tensorflow(activation,X,units):

    model = Sequential(
        [               
            tf.keras.Input(shape=(X.shape[1],)),    #La taille d'entrée
            # Votre code ici -----------------------------------
            Dense(units=25,activation=activation),
            Dense(units=15,activation=activation),
            Dense(units=1,activation=activation),
            # Votre code ici -----------------------------------
        ], name = "my_model" 
    )  
    print(model.summary())
    return model 

In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import pandas as pd

# Prototype for training function with Pandas DataFrame
def train_tensor(nbr_layers, nbr_neurals, activation, alpha, loss, optimizer, epoches, data, target_column):
    """
    nbr_layers: Number of layers (int)
    nbr_neurons: Number of neurons per layer (list of ints) 
    activation: Activation function per layer (list of strings, e.g., 'relu', 'sigmoid')
    alpha: Learning rate (float)
    loss: Loss function (string, e.g., 'mean_squared_error')
    optimizer: Optimizer function (string, e.g., 'adam')
    epochs: Number of epochs (int)
    data: Pandas DataFrame (data) containing features and target column
    target_column: Name of the target column for regression (string)
    """
    # Split the DataFrame into features (X) and target (y)
    X = data.drop(columns=[target_column])  # All columns except the target column
    y = data[target_column]  # Target column for prediction
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # Build the model
    model = models.Sequential()
    
    # Add input layer with the number of features (columns) in X
    model.add(layers.InputLayer(input_shape=(X_train.shape[1],)))
    
    # Add hidden layers
    for i in range(nbr_layers):
        model.add(layers.Dense(units=nbr_neurals[i], activation=activation[i]))
    
    # Add final layer for regression (output a single value)
    model.add(layers.Dense(1))
    
    # Compile the model with the given optimizer and learning rate
    optimizer_func = getattr(tf.keras.optimizers, optimizer)(learning_rate=alpha)
    model.compile(loss=loss, optimizer=optimizer_func)
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=epoches, batch_size=32, validation_split=0.2)
    
    return model, history


In [19]:
# Assuming you have a CSV file and load it into a Pandas DataFrame
data = pd.read_csv('C:/Users/moham/OneDrive/Bureau/detec_fraud/creditcard.csv')

# Define your parameters for the model
nbr_layers = 3
nbr_neurals = [64, 32, 16]
activation = ['relu', 'relu', 'relu']
alpha = 0.001
loss = 'mean_squared_error'
optimizer = 'Adam'
epoches = 10

# Call the function, passing the DataFrame and target column name
model, history = train_tensor(nbr_layers, nbr_neurals, activation, alpha, loss, optimizer, epoches, data, 'Class')


Epoch 1/10


c:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


5697/5697 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 145783.5000 - val_loss: 1.9354
Epoch 2/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 89.9668 - val_loss: 43.5097
Epoch 3/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 481.1045 - val_loss: 0.7906
Epoch 4/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 74.9514 - val_loss: 0.0176
Epoch 5/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 2.4330 - val_loss: 160.1642
Epoch 6/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 827.2957 - val_loss: 7.3347
Epoch 7/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 6s 971us/step - loss: 39.3917 - val_loss: 0.0053
Epoch 8/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 4.6899 - val_loss: 0.0024
Epoch 9/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 0.0073 - val_loss: 0.0024
Epoch 10/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 2.6458 - val_loss: 0.0017


In [20]:
from tensorflow.keras.models import load_model
model.save('my_model.h5') 

In [22]:
!pip install boto3

  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ----------------------

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 479, in read
    s = self.fp.read

In [21]:
import boto3

s3 = boto3.client('s3')
s3.upload_file('aza.h5', 'my-bucket-name', 'my_model.h5')


ModuleNotFoundError: No module named 'boto3'

In [ ]:
import gdown

gdown.download('YOUR_GOOGLE_DRIVE_LINK', 'my_model.h5', quiet=False)
model = load_model('my_model.h5')
s3.download_file('my-bucket-name', 'my_model.h5', 'my_model.h5')
model = load_model('my_model.h5')
